In [11]:
# read iris data
names = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'Class']
X = []
y = []
with open('iris.data') as f:
    f_lines = f.readlines()
    for f_line in f_lines:
        if f_line:
            f_line = f_line.split(',')
            X.append(f_line[:4])
            y.append(["Iris-setosa", "Iris-versicolor", "Iris-virginica"].index(f_line[4].strip()))

In [12]:
X

[['5.1', '3.5', '1.4', '0.2'],
 ['4.9', '3.0', '1.4', '0.2'],
 ['4.7', '3.2', '1.3', '0.2'],
 ['4.6', '3.1', '1.5', '0.2'],
 ['5.0', '3.6', '1.4', '0.2'],
 ['5.4', '3.9', '1.7', '0.4'],
 ['4.6', '3.4', '1.4', '0.3'],
 ['5.0', '3.4', '1.5', '0.2'],
 ['4.4', '2.9', '1.4', '0.2'],
 ['4.9', '3.1', '1.5', '0.1'],
 ['5.4', '3.7', '1.5', '0.2'],
 ['4.8', '3.4', '1.6', '0.2'],
 ['4.8', '3.0', '1.4', '0.1'],
 ['4.3', '3.0', '1.1', '0.1'],
 ['5.8', '4.0', '1.2', '0.2'],
 ['5.7', '4.4', '1.5', '0.4'],
 ['5.4', '3.9', '1.3', '0.4'],
 ['5.1', '3.5', '1.4', '0.3'],
 ['5.7', '3.8', '1.7', '0.3'],
 ['5.1', '3.8', '1.5', '0.3'],
 ['5.4', '3.4', '1.7', '0.2'],
 ['5.1', '3.7', '1.5', '0.4'],
 ['4.6', '3.6', '1.0', '0.2'],
 ['5.1', '3.3', '1.7', '0.5'],
 ['4.8', '3.4', '1.9', '0.2'],
 ['5.0', '3.0', '1.6', '0.2'],
 ['5.0', '3.4', '1.6', '0.4'],
 ['5.2', '3.5', '1.5', '0.2'],
 ['5.2', '3.4', '1.4', '0.2'],
 ['4.7', '3.2', '1.6', '0.2'],
 ['4.8', '3.1', '1.6', '0.2'],
 ['5.4', '3.4', '1.5', '0.4'],
 ['5.2',

In [13]:
y

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2]

In [21]:
# convert to numpy array
import numpy as np
X = np.asarray(X).astype(np.float32)
y = np.asarray(y).astype(np.float32)

In [29]:
# shuffle the dataset
idx = list(range(150))
np.random.seed(17)  
np.random.shuffle(idx) 

In [46]:
# Split the dataset into five parts to do cross-fold validation
idx_1 = idx[:30]
idx_2 = idx[30:60]
idx_3 = idx[60:90]
idx_4 = idx[90:120]
idx_5 = idx[120:]

In [48]:
#The input feature vector is augmented with the 1
X = np.c_[ X, np.ones(150)]

In [49]:
# convert label to vector
t = np.zeros([150,3])
for i in range(150):
    t[i][int(y[i])] = 1

In [210]:
class MLP:
    def __init__(self, samples, labels, size_hidden=20, eta=0.01, epochs=30):
        '''
        :param samples: input samples
        :param labels: input labels
        :param size_hidden: number of units in hidden layer
        :param eta: learning rate
        :param epochs: number of epochs
        '''
        self.samples = samples
        self.labels = labels
        self.size_hidden = size_hidden
        
        # Initialize all weight
        init_range = (6 / (len(self.samples[0]) + 1 + len(self.labels[0]))) ** 0.5
        self.w01 = 2 * init_range * np.random.random((len(self.samples[0]), size_hidden)) - init_range
        self.w12 = 2 * init_range * np.random.random((size_hidden, len(self.labels[0]))) - init_range
                      
        self.eta = eta
        self.epochs = epochs

    def tanh(self, x, deriv=False):
        if (deriv == True):
            return 1.0 - np.tanh(x)**2
        return np.tanh(x)

    def fit(self):
        accuracy = []
        no_epochs = []
        sample_no = 0
        
        for epoch in range(self.epochs):
            # shuffle train data 
            idx_sample = list(range(len(self.samples[0])))
            np.random.shuffle(idx_sample)
            self.samples = self.samples[idx_sample]
            self.labels = self.labels[idx_sample]
            
            for i in range(0, len(self.samples), 1):
                sample_no += 1
                x = self.samples[i]
                t = self.labels[i]

                # forward propagate
                a = np.dot(x, self.w01)
                z = self.tanh(a)
                y = np.dot(z, self.w12)

                error = ((1 / 2) * np.power((y - t), 2))
                error_total = str(np.mean(np.abs(error)))

                # Backpropagation
                y_delta = y - t
                y_delta = np.dot(y_delta.reshape(1, len(self.labels[0])).T, z.reshape(1, self.size_hidden))

                z_delta = np.dot(self.w12, y_delta)
                z_delta = np.dot(self.tanh(z, deriv=True).reshape(1, self.size_hidden), z_delta)
                z_delta = np.dot(z_delta.T, x.reshape(1, len(self.samples[0])))

                # Update Weights
                self.w01 -= self.eta * z_delta.T
                self.w12 -= self.eta * y_delta.T
                      
            print("Epoch: ", epoch, " - Error: ", error_total)
            print("############################################")

    def predict(self, test_samples, test_labels):
        a = np.dot(test_samples, self.w01)
        z = self.tanh(a)
        y = np.dot(z, self.w12)
        y_pred = (y == y.max(axis=1)[:, None]).astype(float)
        
        res_pred = []
        res_labels = []

        for k in y_pred:
            for i, j in enumerate(k):
                if int(j) == 1:
                    res_pred.append(i)
                    break
        for k in test_labels:
            for i, j in enumerate(k):
                if j == 1.0:
                    res_labels.append(i)

        return res_pred, res_labels

In [211]:
mlp = MLP(X[idx_1+idx_2+idx_3+idx_4], t[idx_1+idx_2+idx_3+idx_4])

In [212]:
mlp.fit()

Epoch:  0  - Error:  2.8017343887260933
############################################
Epoch:  1  - Error:  0.8687112832576648
############################################
Epoch:  2  - Error:  0.14775287529046635
############################################
Epoch:  3  - Error:  0.13132098462752093
############################################
Epoch:  4  - Error:  0.11666476009835224
############################################
Epoch:  5  - Error:  0.10761741152409304
############################################
Epoch:  6  - Error:  0.22214337825480254
############################################
Epoch:  7  - Error:  0.10033075907615195
############################################
Epoch:  8  - Error:  0.2133016560202685
############################################
Epoch:  9  - Error:  0.13970476350787617
############################################
Epoch:  10  - Error:  0.20298249748925276
############################################
Epoch:  11  - Error:  0.10053185926353098
##############

In [217]:
y_pred, y_true = mlp.predict(X[idx_5], t[idx_5])

In [216]:
from sklearn.metrics import accuracy_score

In [218]:
print("Accuracy: ",accuracy_score(y_true, y_pred))

Accuracy:  0.4


In [108]:
def compute_wine_data(filename, order_file):
    """
    construct data for training and testing using the order_file

    Args:
        filename: Result filename.
        order_file: Orders filename.

    Returns:

    """

    def convert_size_to_int(_size):
        """

        Args:
            _size:

        Returns:

        """
        default_size = 0

        if isinstance(_size, str):
            if 'none' in _size.lower():
                return default_size

            if 'nan' in _size.lower():
                return default_size

            return int(_size)

        if np.isnan(_size):
            return default_size

        return int(_size)

    def convert_quantity_to_int(quant):
        """

        Args:
            quant:

        Returns:

        """
        if quant == 0:
            return 0

        if np.isnan(quant):
            return 0

        return int(quant)

    Orders = pd.read_csv(order_file)
    Orders['Size'] = Orders['Size'].apply(convert_size_to_int)
    Orders['Quantity'] = Orders['Quantity'].apply(convert_quantity_to_int)
    Orders['ImplicitFb'] = Orders['Quantity'] * Orders['Size'] / 1000

    orders_by_custid = Orders.groupby('customerId')

    ratings = []
    users = []
    items = []
    wine_idx = 0
    cust_id_mapping = {}
    wine_id_mapping = {}

    for cust_idx, (customer_id, gr) in enumerate(orders_by_custid):
        cust_id_mapping[customer_id] = cust_idx

        wine_id_and_ratings = gr[['wineId', 'ImplicitFb']].to_dict('index')
        unique_wine_ratings = {}

        for _wine_id_and_rating in wine_id_and_ratings.values():

            _wine_id = _wine_id_and_rating['wineId']
            _rating = _wine_id_and_rating['ImplicitFb']
            # check duplicate wineID
            if _wine_id not in unique_wine_ratings:
                unique_wine_ratings[_wine_id] = _rating

            else:
                unique_wine_ratings[_wine_id] += _rating

        for w_id, _rating in unique_wine_ratings.items():
            ratings.append(_rating)
            items.append(w_id)
            if w_id not in wine_id_mapping:
                wine_id_mapping[w_id] = wine_idx
                wine_idx += 1

        users.extend([customer_id] * len(unique_wine_ratings))

        # if cust_idx > 5:
        #     break

    with open(filename, 'wb') as f:
        wine_ids = np.array(list(wine_id_mapping.keys()))
        cust_ids = np.array(list(cust_id_mapping.keys()))
        row_idx = [wine_id_mapping[i] for i in items]
        col_idx = [cust_id_mapping[i] for i in users]
        wine_cust_rating_mat = csr_matrix((ratings, (row_idx, col_idx)),
                                          shape=(len(wine_ids), len(cust_ids)))

        pickle.dump((wine_ids, cust_ids, wine_cust_rating_mat), f)

In [117]:
compute_wine_data('jjb1.pkl', 'ai-orders.csv')

In [110]:
compute_wine_data('jjb2.pkl', 'ai-orders.csv')

In [53]:
for cust_idx, (customer_id, gr) in enumerate(orders_by_custid):
    cust_id_mapping[customer_id] = cust_idx

    wine_id_and_ratings = gr[['wineId', 'ImplicitFb']].to_dict('index')
    unique_wine_ratings = {}

    for _wine_id_and_rating in wine_id_and_ratings.values():

        _wine_id = _wine_id_and_rating['wineId']
        _rating = _wine_id_and_rating['ImplicitFb']
        # check duplicate wineID
        if _wine_id not in unique_wine_ratings:
            unique_wine_ratings[_wine_id] = _rating

        else:
            unique_wine_ratings[_wine_id] += _rating

    for w_id, _rating in unique_wine_ratings.items():
        ratings.append(_rating)
        items.append(w_id)
        if w_id not in wine_id_mapping:
            wine_id_mapping[w_id] = wine_idx
            wine_idx += 1

    users.extend([customer_id] * len(unique_wine_ratings))


In [69]:
with open('jjb1.pkl', 'wb') as f:
    wine_ids = np.array(list(wine_id_mapping.keys()))
    cust_ids = np.array(list(cust_id_mapping.keys()))
    row_idx = [wine_id_mapping[i] for i in items]
    col_idx = [cust_id_mapping[i] for i in users]

    pickle.dump((wine_ids, cust_ids), f)


In [70]:
with open('jjb2.pkl', 'wb') as f:
    wine_ids = np.array(list(wine_id_mapping.keys()))
    cust_ids = np.array(list(cust_id_mapping.keys()))
    row_idx = [wine_id_mapping[i] for i in items]
    col_idx = [cust_id_mapping[i] for i in users]

    pickle.dump((wine_ids, cust_ids), f)

In [66]:
row_idx

[0,
 1,
 2,
 3,
 4,
 5,
 6,
 7,
 8,
 9,
 10,
 11,
 7,
 12,
 13,
 14,
 5,
 15,
 8,
 16,
 15,
 11,
 17,
 8,
 14,
 18,
 15,
 10,
 11,
 17,
 12,
 8,
 5,
 19,
 16,
 17,
 17,
 20,
 8,
 21,
 22,
 23,
 24,
 25,
 12,
 13,
 26,
 27,
 8,
 14,
 18,
 16,
 10,
 11,
 17,
 7,
 25,
 12,
 13,
 28,
 8,
 14,
 5,
 9,
 18,
 19,
 16,
 15,
 4,
 29,
 30,
 31,
 32,
 20,
 7,
 12,
 33,
 34,
 35,
 17,
 25,
 13,
 36,
 37,
 38,
 39,
 5,
 7,
 4,
 40,
 11,
 17,
 30,
 41,
 7,
 42,
 25,
 12,
 38,
 13,
 36,
 43,
 28,
 8,
 14,
 44,
 5,
 9,
 18,
 45,
 15,
 46,
 47,
 48,
 8,
 49,
 25,
 36,
 50,
 51,
 25,
 4,
 30,
 42,
 12,
 39,
 19,
 4,
 52,
 29,
 11,
 37,
 17,
 31,
 20,
 7,
 25,
 38,
 33,
 13,
 39,
 53,
 43,
 28,
 54,
 55,
 8,
 14,
 35,
 44,
 5,
 56,
 18,
 45,
 19,
 16,
 15,
 46,
 21,
 57,
 10,
 58,
 59,
 60,
 61,
 62,
 63,
 64,
 65,
 66,
 67,
 8,
 68,
 69,
 70,
 71,
 72,
 73,
 74,
 75,
 76,
 77,
 78,
 37,
 79,
 16,
 80,
 81,
 82,
 4,
 83,
 84,
 85,
 86,
 87,
 88,
 89,
 90,
 91,
 92,
 93,
 94,
 95,
 96,
 97,
 98,
 99,
 100

In [50]:
wine_id_mapping.keys()

dict_keys(['2006-842-750', '2019-11953-375', '2019-6899-750', '9998-12032-750', '2006-13126-750', '2018-210454-750', '2019-10949-750', '2016-2076-750', '2018-12836-750', '2018-3666-750', '2001-198579-750', '2015-213821-750', '2016-304630-1500', '2017-10147-750', '2018-14220-750', '2019-212156-750', '2019-207394-750', '2016-10485-750', '2018-4830-750', '2019-15384-750', '2016-2050-750', '2019-2790-750', '2012-196170-750', '2015-185-750', '2015-2081-750', '2016-2322-750', '2017-12575-750', '2017-198557-750', '2017-339148-750', '2014-330950-750', '2016-11046-750', '2016-13947-750', '2016-186673-750', '2016-805-750', '2018-11612-750', '2018-14711-750', '2017-1462-750', '2015-959-750', '2016-4008-1500', '2017-10204-750', '2007-214823-750', '2016-206468-750', '2016-215602-750', '2017-215627-750', '2018-14953-750', '2019-14545-750', '2019-216714-750', '2016-1281-750', '2017-8995-750', '2019-199574-750', '2016-211737-750', '2016-387-750', '2006-209548-375', '2017-213739-750', '2018-11074-750',

In [55]:
items

['2006-842-750',
 '2019-11953-375',
 '2019-6899-750',
 '9998-12032-750',
 '2006-13126-750',
 '2018-210454-750',
 '2019-10949-750',
 '2016-2076-750',
 '2018-12836-750',
 '2018-3666-750',
 '2001-198579-750',
 '2015-213821-750',
 '2016-2076-750',
 '2016-304630-1500',
 '2017-10147-750',
 '2018-14220-750',
 '2018-210454-750',
 '2019-212156-750',
 '2018-12836-750',
 '2019-207394-750',
 '2019-212156-750',
 '2015-213821-750',
 '2016-10485-750',
 '2018-12836-750',
 '2018-14220-750',
 '2018-4830-750',
 '2019-212156-750',
 '2001-198579-750',
 '2015-213821-750',
 '2016-10485-750',
 '2016-304630-1500',
 '2018-12836-750',
 '2018-210454-750',
 '2019-15384-750',
 '2019-207394-750',
 '2016-10485-750',
 '2016-10485-750',
 '2016-2050-750',
 '2018-12836-750',
 '2019-2790-750',
 '2012-196170-750',
 '2015-185-750',
 '2015-2081-750',
 '2016-2322-750',
 '2016-304630-1500',
 '2017-10147-750',
 '2017-12575-750',
 '2017-198557-750',
 '2018-12836-750',
 '2018-14220-750',
 '2018-4830-750',
 '2019-207394-750',
 '20

In [ ]:
items